In [38]:
import pandas as pd

In [39]:
import requests
import json

In [63]:
def callApi(method, stock):
    key = "?token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    url = host + ticker + api_method + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [80]:
callApi("balance-sheet", "LUV")

{'symbol': 'LUV',
 'balancesheet': [{'reportDate': '2020-06-30',
   'fiscalDate': '2020-06-30',
   'currency': 'USD',
   'currentCash': 12351000000,
   'shortTermInvestments': 2106000000,
   'receivables': 770000000,
   'inventory': 422000000,
   'otherCurrentAssets': 223000000,
   'currentAssets': 15872000000,
   'longTermInvestments': 68000000,
   'propertyPlantEquipment': 18092000000,
   'goodwill': 970000000,
   'intangibleAssets': 295000000,
   'otherAssets': 299000000,
   'totalAssets': 35596000000,
   'accountsPayable': 336000000,
   'currentLongTermDebt': 695000000,
   'otherCurrentLiabilities': 7922000000,
   'totalCurrentLiabilities': 9361000000,
   'longTermDebt': 10391000000,
   'otherLiabilities': 2553000000,
   'minorityInterest': 0,
   'totalLiabilities': 24718000000,
   'commonStock': 589872108,
   'retainedEarnings': 16842000000,
   'treasuryStock': 10881000000,
   'capitalSurplus': 4159000000,
   'shareholderEquity': 10878000000,
   'netTangibleAssets': 9908000000}]}

In [57]:
def peRatio(quote):
    if not("peRatio" in quote.keys()):
        return 0
    return quote["peRatio"]

In [66]:
def priceToBook(stats):
    if not("priceToBook" in stats.keys()):
        return 0
    return stats["priceToBook"]

In [72]:
def pegRatio(stats):
    if not("debtToEquity" in stats.keys()):
        return 0
    return stats["debtToEquity"]

In [78]:
def debtToEquity(stats):
    
    if not("pegRatio" in stats.keys()):
                return 0
    return stats["pegRatio"]

In [84]:
# Current Ratio= Current Assets / Current Liabilities
def currentRatio(balance_sheet):
    if not("balancesheet" in balance_sheet.keys()) or len(balance_sheet["balancesheet"]) == 0:
        return 0
    latest_balance_sheet = balance_sheet["balancesheet"][0]
    if not("totalAssets" in latest_balance_sheet.keys()) and not("totalLiabilities" in latest_balance_sheet.keys()):
        return 0
    return latest_balance_sheet["totalAssets"]/ latest_balance_sheet["totalLiabilities"]
    

In [85]:
tickers = ['LUV','TELA', 'FB', 'MAR', 'JPM','HTZ', 'DAL' ]
dataSet = [] 
for ticker in tickers:
    data = {}
    points = 0
    data["Ticker"] = ticker
    
    # Major data set 
    quote = callApi("quote", ticker)
    stats = callApi("advanced-stats", ticker)
    balance_sheet = callApi("balance-sheet", ticker)
    
    # PE Ratio < 15
    pe_ratio = peRatio(quote)
    if pe_ratio != 0: 
        data["PE Ratio < 15"] = pe_ratio
    else: 
        data["PE Ratio < 15"] = "NA"

    # Price To Book Ratio  
    price_to_book = priceToBook(stats)
    if price_to_book != 0: 
        data["Price To Book < 1.5"] = price_to_book
    else: 
        data["Price To Book < 1.5"] = "NA"
    
    if pe_ratio * price_to_book < 22.5 and pe_ratio * price_to_book > 0: 
        points += 1
    
    # PEG Ratio 
    peg_ratio = pegRatio(stats)
    if peg_ratio != 0: 
        data["PEG Ratio(1 and 0)"] = peg_ratio
    else: 
        data["PEG Ratio(1 and 0)"] = "NA"
    if peg_ratio < 1.1 and peg_ratio > 0:
        points += 1
    
    # Debt To Equity Ratio 
    # Warent buffet say < 0.5
    debt_Equity = debtToEquity(stats)
    if debt_Equity != 0: 
        data["Debt to Equity (0 and 1.5)"] = debt_Equity
    else: 
        data["Debt to Equity (0 and 1.5)"] = "NA"
    if debt_Equity < 1.6 and debt_Equity > 0:
        points += 1
        
    # Current Ratio 
    current_ratio = currentRatio(balance_sheet)
    if current_ratio != 0: 
        data["Current Ratio > 1.5"] = current_ratio
    else: 
        data["Current Ratio > 1.5"] = "NA"
        
    if current_ratio > 1.4:
        points += 1
    
    data["Points"] = points
    dataSet.append(data.copy())
pd.DataFrame(dataSet)    

,Ticker,PE Ratio < 15,Price To Book < 1.5,PEG Ratio(1 and 0),Debt to Equity (0 and 1.5),Current Ratio > 1.5,Points
0,LUV,96.62,2.363208,-0.33,-0.33,1.440084,1
1,TELA,-5.48,6.140537,0.43,0.43,2.856766,3
2,FB,34.29,6.726721,0.35,0.35,4.776741,3
3,MAR,74.81,48.356612,-0.53,-0.53,0.996933,0
4,JPM,13.92,1.207238,-0.27,-0.27,1.089691,1
5,HTZ,-0.18,0.127155,NA,NA,1.029987,0
6,DAL,-5.46,1.319489,0.02,0.02,1.136698,2
